# Time- and energy resolved effects in thermal neutrons detectors
> __Author:__ A. Backis
<br/>__Institute:__ European Spallation Source (ESS), University of Glasgow
<br/>__Date:__ 7/1-2020

_Abstract:_
This notebook contains the data-analysis tools used for the corresponding paper. It describes how the analysis was performed on data obtained with the Multi-Grid detector at the HZB.

## Contents
* [1. Introduction](#INTRODUCTION)
    * [1.1 Overview](#OVERVIEW)
    * [1.2 Packages](#PACKAGES)
* [2. Data](#DATA)
    * [2.1 Extract](#EXTRACT)
    * [2.2 Load](#LOAD)
    * [2.3 Filter](#FILTER)
* [3. Visualization](#VISUALIZATION)
    * [3.1 PHS](#PHS)
        * [3.1.1 1D](#PHS_1D)
        * [3.1.2 2D](#PHS_2D)
        * [2.1.3 Wires vs grids](#PHS_WIRES_VS_GRIDS)
    * [3.2 Time-of-flight](#TIME_OF_FLIGHT)
    * [3.3 Coincidences](#COINCIDENCES)
        * [3.3.1 Wires vs grids](#COINCIDENCES_WIRES_VS_GRIDS)
        * [3.3.2 Projections](#PROJECTIONS)
        * [3.3.3 3D-Rotation](#3D_ROTATION)
    * [3.4 Multiplicity](#MULTIPLICITY)
    * [3.5 Rate](#RATE)
    * [3.6 Layers](#LAYERS)
    * [3.7 Energy](#ENERGY)
    * [3.8 Wavelength](#WAVELENGTH)
* [4. Analysis](#ANALYSIS)
    * [4.1 Electronic noise](#ELECTRONIC_NOISE)
    * [4.2 Efficiency](#EFFICIENCY)
    * [4.3 Line shape](#LINE_SHAPE)
    * [4.4 Time- and energy resolution](#TIME_AND_ENERGY_RESOLUTION)
* [Acknowledgements](#ACKNOWLEDGEMENTS)
* [References](#REFERENCES)

### 1. Introduction<a class="anchor" id="INTRODUCTION"></a>

#### 1.1 Overview<a class="anchor" id="OVERVIEW"></a>

#### 1.2 Packages<a class="anchor" id="PACKAGES"></a>

In [ ]:
# Autoload packages when doing an external change
%load_ext autoreload
%autoreload 2

# Make matplotlib interactive mode in notebook
%matplotlib inline
%matplotlib notebook

# General packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Data handling
import mg.file_handling.read as mg_read
import he3.read as he3_read

# Plotting
import mg.plotting.basic_plot as mg_basic_plot
import mg.plotting.advanced_plot as mg_advanced_plot
import he3.plotting as he3_plot

# Energy calculation
import mg.helper_functions.energy_calculation as mg_energy
import he3.energy_calculation as he3_energy

# Helper functions
import mg.helper_functions.misc as mg_hf

Declare helper functions and global variables

In [ ]:

# Declare beam monitors norms
MONITOR_NORM_R = 1/11411036
MONITOR_NORM_NR = 1/9020907
MONITOR_NORM_HE3 = 1/10723199

# Declare distance offsets
DIST_OFFSET_R = 1.5e-3
DIST_OFFSET_NR = 0
DIST_OFFSET_HE3 = 3e-3

def set_thick_labels(thickness):
    # Customize matplotlib font sizes
    plt.rc('font', size=thickness)          # controls default text sizes
    plt.rc('axes', titlesize=thickness)     # fontsize of the axes title
    plt.rc('axes', labelsize=thickness)    # fontsize of the x and y labels
    plt.rc('xtick', labelsize=thickness)    # fontsize of the tick labels
    plt.rc('ytick', labelsize=thickness)    # fontsize of the tick labels
    plt.rc('legend', fontsize=thickness)    # legend fontsize
    plt.rc('figure', titlesize=thickness)  # fontsize of the figure title

### 2. Data<a class="anchor" id="DATA"></a>

In [ ]:
# Declare paths to raw data
mg_radial_path = '../data/raw/mvmelst_165_191002_111641_Det2_overnight3.zip'
mg_no_radial_path = '../data/raw/mvmelst_135_190930_141618_Det1_overnight2_30x80_14x60.zip'
he3_path = '../data/raw/2019_09_HZB_He3InBeam54304s_overnight.lst'
# Declare paths to clusters
mg_radial_clusters_path = '../data/processed/clusters_mvmelst_165_191002_111641_Det2_overnight3.h5'
mg_no_radial_clusters_path = '../data/processed/clusters_mvmelst_135_190930_141618_Det1_overnight2_30x80_14x60.h5'
# Declare paths to events
mg_radial_events_path = '../data/processed/events_mvmelst_165_191002_111641_Det2_overnight3.h5'
mg_no_radial_events_path = '../data/processed/events_mvmelst_135_190930_141618_Det1_overnight2_30x80_14x60.h5'
he3_events_path = '../data/processed/events_2019_09_HZB_He3InBeam54304s_overnight.h5'

#### 2.1 Extract<a class="anchor" id="EXTRACT"></a>

In [ ]:
def extract_mg_data(zipped_path, clusters_save_path, events_save_path):
    unzipped_path = mg_read.unzip_data(zipped_path)
    data = mg_read.import_data(unzipped_path)
    # Extract clusters and save to disc
    clusters = mg_read.extract_clusters(data)
    mg_read.save_data(clusters, clusters_save_path)
    clusters = None
    # Extract events and save to disc
    #events = mg_read.extract_events(data)
    #mg_read.save_data(events, events_save_path)
    events = None
    # Clear data
    data = None

In [ ]:
def extract_he3_data(raw_path, save_path):
    df = he3_read.extract_events(raw_path)
    he3_read.save_data(df, save_path)
    df = None

In [ ]:
extract_mg_data(mg_radial_path, mg_radial_clusters_path, mg_radial_events_path)
extract_mg_data(mg_no_radial_path, mg_no_radial_clusters_path, mg_no_radial_events_path)
#extract_he3_data(he3_path, he3_events_path)

#### 2.2 Load<a class="anchor" id="LOAD"></a>

In [ ]:
# Load multi-grid clusters
mg_radial_ce_df = mg_read.load_data(mg_radial_clusters_path)
mg_no_radial_ce_df = mg_read.load_data(mg_no_radial_clusters_path)
# Load multi-grid events
mg_radial_e_df = mg_read.load_data(mg_radial_events_path)
mg_no_radial_e_df = mg_read.load_data(mg_no_radial_events_path)
# Load He-3 events
he3_df = he3_read.load_data(he3_events_path)

#### 2.3 Filter<a class="anchor" id="FILTER"></a>

In [ ]:
"""
Filters are declared in the following format:
{'PARAMETER': [MIN_VALUE, MAX_VALUE, IS_ACTIVATE]}
"""

us_to_tdc = 1 / (62.5e-9 * 1e6)

# Declare filter for multi-grid clusters
mg_filter_clusters = {'wm': [1, 1, True],                   # Wire multiplicity
                      'gm': [1, 5, True],                   # Grid multiplicity
                      'wadc': [600, np.inf, True],             # Wire charge
                      'gadc': [600, np.inf, True],             # Grid charge
                      'tof': [0, np.inf, True],           # Time-of-flight (TDC channels)
                      'time': [0, np.inf, True],            # Time (TDC channels)
                      'bus': [0, 2, True],                  # Bus
                      'flag': [0, 1, False],                # =1 if different buses within same coincidence
                      'layer': [0, 19, False],              # Layer, front=0 to back=19
                      'row': [0, 11, False],                # Row, right to left (seen from neutrons)
                      'gch': [80, 119, True]}              # Grid channel, bottom=80 to top=119

# Declare filter for multi-grid events
mg_filter_events = {'bus': [0, 2, False],                   # Bus
                    'ch': [0, 119, False],                  # Channel
                    'adc': [0, np.inf, False]}              # Collected charge (ADC channels)

# Declare filters for He-3 data
he3_filter = {'adc': [7500, np.inf, True],                  # Collected charge (ADC channels)
              'tof': [0, 71429e-6, True]}                   # Time-of-flight (s)

# Perform filters
mg_r = mg_read.filter_data(mg_radial_ce_df, mg_filter_clusters)
mg_nr = mg_read.filter_data(mg_no_radial_ce_df, mg_filter_clusters)
he3 = he3_read.filter_data(he3_df, he3_filter)

### 3. Visualization<a class="anchor" id="VISUALIZATION"></a>

#### 3.1 PHS<a class="anchor" id="PHS"></a>

##### 3.1.1 PHS - 1D<a class="anchor" id="PHS_1D"></a>

In [ ]:
%matplotlib notebook
# Define parameters
number_bins = 500
interval = [0, 1e4]
# Plot
fig = mg_basic_plot.phs_1d_plot(mg_r, number_bins, interval, 'Radial coating')
fig.show()
fig = mg_basic_plot.phs_1d_plot(mg_nr, number_bins, interval, 'No radial coating')
fig.show()
fig = he3_plot.phs_1d_plot(he3, number_bins, 'He-3')
fig.show()

##### 3.1.2 PHS - 2D<a class="anchor" id="PHS_2D"></a>

In [ ]:
# Define parameters
bus_start = 0
bus_stop = 2
# Plot
fig = mg_basic_plot.phs_2d_plot(mg_radial_e_df, bus_start, bus_stop, 'Radial coating')
fig.show()
fig = mg_basic_plot.phs_2d_plot(mg_no_radial_e_df, bus_start, bus_stop, 'No radial coating')
fig.show()

##### 3.1.3 PHS - Wires vs grids<a class="anchor" id="PHS_WIRES_VS_GRIDS"></a>

In [ ]:
# Define parameters
bus_start = 0
bus_stop = 2
# Plot
fig = mg_basic_plot.phs_wires_vs_grids_plot(mg_r, bus_start, bus_stop, 'Radial coating')
fig.show()
fig = mg_basic_plot.phs_wires_vs_grids_plot(mg_nr, bus_start, bus_stop, 'No radial coating')
fig.show()

#### 3.2 Time-of-flight<a class="anchor" id="TIME_OF_FLIGHT"></a>

In [ ]:
%matplotlib notebook
# Declare parameters
number_bins = 2000
interval = [0, 71429]
# Plot
fig = plt.figure()
mg_basic_plot.tof_histogram(mg_r, number_bins, 'Radial coating')
fig.show()
fig = plt.figure()
mg_basic_plot.tof_histogram(mg_nr, number_bins, 'No radial coating')
fig.show()
fig = plt.figure()
he3_plot.tof_histogram(he3, number_bins, 'He-3')
fig.show()
fig = plt.figure()
mg_basic_plot.tof_histogram(mg_r, number_bins, '', label='Radial', interval=interval)
mg_basic_plot.tof_histogram(mg_nr, number_bins, 'Overlay', label='No radial', interval=interval)
he3_plot.tof_histogram(he3, number_bins, label='He-3', interval=interval)
plt.legend()
fig.show()

#### 3.3 Coincidences<a class="anchor" id="COINCIDENCES"></a>

##### 3.3.1 Wires vs grids<a class="anchor" id="COINCIDENCES_WIRES_VS_GRIDS"></a>

In [ ]:
# Declare parameters
duration_r = mg_r.time.values[-1] - mg_r.time.values[0]
duration_nr = mg_nr.time.values[-1] - mg_nr.time.values[0]
bus_start = 0
bus_stop = 2
# Plot
fig, __ = mg_basic_plot.ce_2d_plot(mg_r, duration_r, bus_start, bus_stop, 'Radial coating')
fig.show()
fig, __ = mg_basic_plot.ce_2d_plot(mg_nr, duration_nr, bus_start, bus_stop, 'No radial coating')
fig.show()

##### 3.3.2 Projections<a class="anchor" id="PROJECTIONS"></a>

In [ ]:
# Declare parameters
bus_start = 0
bus_stop = 2
# Plot
fig, __ = mg_basic_plot.ce_projections_plot(mg_r, bus_start, bus_stop, 'Radial coating')
fig.show()
fig, __ = mg_basic_plot.ce_projections_plot(mg_nr, bus_start, bus_stop, 'No radial coating')
fig.show()

##### 3.3.3 3D-rotation<a class="anchor" id="3D_ROTATION"></a>

In [ ]:
mg_basic_plot.ce_3d_plot(mg_r, 'Radial coating')
mg_basic_plot.ce_3d_plot(mg_nr, 'No radial coating')

#### 3.4 Multiplicity<a class="anchor" id="MULTIPLICITY"></a>

In [ ]:
# Declare parameters
bus_start = 0
bus_stop = 2
# Plot
fig = mg_basic_plot.multiplicity_plot(mg_r, bus_start, bus_stop, 'Radial coating')
fig.show()
fig = mg_basic_plot.multiplicity_plot(mg_nr, bus_start, bus_stop, 'No radial coating')
fig.show()

#### 3.5 Rate<a class="anchor" id="RATE"></a>

In [ ]:
# Declare parameters
number_bins = 50
# Plot
fig = plt.figure()
mg_basic_plot.rate_plot(mg_r, number_bins, 'Radial coating')
mg_basic_plot.rate_plot(mg_nr, number_bins, 'No radial coating')
plt.legend()
fig.show()

##### 3.6 Layers<a class="anchor" id="LAYERS"></a>

###### 3.6.1 Time-of-flight, voxel by voxel

In [ ]:
#fig = mg_advanced_plot.layers_tof(mg_r, 'Multi-Grid:R')
#fig.show()
set_thick_labels(10)
fig = mg_advanced_plot.layers_tof(mg_nr, 'Multi-Grid:NR')
fig.show()

###### 3.6.2 Counts, layer by layer

In [ ]:
# Declare parameters
duration_r = mg_r.time.values[-1] - mg_r.time.values[0]
duration_nr = mg_nr.time.values[-1] - mg_nr.time.values[0]
# Plot
fig = mg_advanced_plot.layers_counts(mg_r, duration_r, 'Coated radial')
fig.show()
fig = mg_advanced_plot.layers_counts(mg_nr, duration_nr, 'Not coated radial')
fig.show()

##### 3.7 Energy<a class="anchor" id="ENERGY"></a>

In [ ]:
# Declare distance offsets
dist_offset_r = 1.5e-3
dist_offset_nr = 0
dist_offset_he3 = 3e-3

# Calculate energy distribution
energies_r = mg_energy.get_energies(mg_r, dist_offset_r)
energies_nr = mg_energy.get_energies(mg_nr, dist_offset_nr)
energies_he3 = he3_energy.get_energies(he3, dist_offset_he3)

# Declare parameters
number_bins = 10000
start = 0.8 # Å
stop = 10 # Å

# Plot
fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(14)
__, __ = mg_advanced_plot.energy_plot(energies_r, number_bins, 'Radial coating',
                                      useMaxNorm=True, start=start, stop=stop)
__, __ = mg_advanced_plot.energy_plot(energies_nr, number_bins, 'No radial coating',
                                      useMaxNorm=True, start=start, stop=stop)
__, __ = he3_plot.energy_plot(energies_he3, number_bins, 'Helium-3 tube', 
                              useMaxNorm=True, start=start, stop=stop)
plt.legend()
fig.show()

##### 3.8 Wavelength<a class="anchor" id="WAVELENGTH"></a>

In [ ]:
# Calculate energy distribution
energies_r = mg_energy.get_energies(mg_r, DIST_OFFSET_R)
energies_nr = mg_energy.get_energies(mg_nr, DIST_OFFSET_NR)
energies_he3 = he3_energy.get_energies(he3, DIST_OFFSET_HE3)

# Declare parameters
number_bins = 50000
start = 0.5
stop = 10

# Plot
fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(14)
__, __ = mg_advanced_plot.wavelength_plot(energies_r, number_bins, 'Radial coating',
                                          useMaxNorm=True, start=start, stop=stop)
__, __ = mg_advanced_plot.wavelength_plot(energies_nr, number_bins, 'No radial coating',
                                          useMaxNorm=True, start=start, stop=stop)
__, __ = he3_plot.wavelength_plot(energies_he3, number_bins, 'Helium-3 tube',
                                  useMaxNorm=True, start=start, stop=stop)
plt.legend()
fig.show()
fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(14)
__, __ = mg_advanced_plot.wavelength_plot(energies_r, number_bins, 'Radial coating',
                                          useMaxNorm=True, start=start, stop=stop)
__, __ = mg_advanced_plot.wavelength_plot(energies_nr, number_bins, 'No radial coating', 
                                          useMaxNorm=True, start=start, stop=stop)
__, __ = he3_plot.wavelength_plot(energies_he3, number_bins, 'Helium-3 tube',
                                  useMaxNorm=True, start=start, stop=stop)
plt.legend()
plt.yscale('log')
fig.show()

In [ ]:
# =============================================================================
#                                 GET ALL PEAKS - WAVELENGTH
# =============================================================================

# Calculate energy distribution
energies_r = mg_energy.get_energies(mg_r, DIST_OFFSET_R)
energies_nr = mg_energy.get_energies(mg_nr, DIST_OFFSET_NR)
energies_he3 = he3_energy.get_energies(he3, DIST_OFFSET_HE3)

# Define peak locations (here we use the He-3 peak locations)
peaks = np.array([0.886, 1.030, 1.177, 1.321, 1.466, 1.610, 1.755, 1.900, 2.044, 2.188, 
                  2.333, 2.477, 2.622, 2.766, 2.911, 3.056, 3.200, 3.344, 3.489, 3.633,
                  3.778, 3.923, 4.068, 4.211, 4.356, 4.500, 4.645, 4.790, 4.934, 5.079,
                  5.223, 5.368, 5.513, 5.657, 5.801, 5.946, 6.090, 6.235, 6.379, 6.524,
                  6.669, 6.813, 6.958, 7.102, 7.246, 7.391, 7.536, 7.680, 7.825, 7.969])
peak_window = 0.04

# Declare plotting specifications
number_bins = 250
lower_y = 0
upper_y = 40e3

# Iterate through all peaks
for peak in peaks:
    # Plot all data together
    fig = plt.figure()
    hist_r, bins_r = mg_advanced_plot.wavelength_plot(energies_r, number_bins, 'Multi-Grid:R',
                                                      start=peak-peak_window/2, stop=peak+peak_window/2,
                                                      color='green', scaling=MONITOR_NORM_R)
    hist_nr, bins_nr = mg_advanced_plot.wavelength_plot(energies_nr, number_bins, 'Multi-Grid:NR',
                                                        start=peak-peak_window/2, stop=peak+peak_window/2,
                                                        color='blue', scaling=MONITOR_NORM_NR)
    hist_he3, bins_he3 = he3_plot.wavelength_plot(energies_he3, number_bins, 'Helium-3',
                                                  start=peak-peak_window/2, stop=peak+peak_window/2,
                                                  color='red', scaling=MONITOR_NORM_HE3)
    plt.legend(title='Detector')
    plt.title('Wavelength distribution, peak at %.3f Å (%.3f meV)' % (peak, mg_hf.A_to_meV(peak)))
    plt.ylabel('Normalized counts')
    plt.locator_params(axis='y', nbins=5)
    plt.locator_params(axis='x', nbins=5)
    #plt.yscale('log')
    # Save data
    output_path = '../output/peak_at_%.2f_Å_%.2f_meV.pdf' % (peak, mg_hf.A_to_meV(peak))
    fig.savefig(output_path, bbox_inches='tight')
    plt.close()

In [ ]:
# =============================================================================
#                                 GET ALL PEAKS - ENERGY
# =============================================================================

# Calculate energy distribution
energies_r = mg_energy.get_energies(mg_r, DIST_OFFSET_R)
energies_nr = mg_energy.get_energies(mg_nr, DIST_OFFSET_NR)
energies_he3 = he3_energy.get_energies(he3, DIST_OFFSET_HE3)

# Define peak locations (here we use the He-3 peak locations)
peaks = np.array([0.886, 1.030, 1.177, 1.321, 1.466, 1.610, 1.755, 1.900, 2.044, 2.188, 
                  2.333, 2.477, 2.622, 2.766, 2.911, 3.056, 3.200, 3.344, 3.489, 3.633,
                  3.778, 3.923, 4.068, 4.211, 4.356, 4.500, 4.645, 4.790, 4.934, 5.079,
                  5.223, 5.368, 5.513, 5.657, 5.801, 5.946, 6.090, 6.235, 6.379, 6.524,
                  6.669, 6.813, 6.958, 7.102, 7.246, 7.391, 7.536, 7.680, 7.825, 7.969])
peak_window = 0.04

# Declare plotting specifications
number_bins = 250
lower_y = 0
upper_y = 40e3

# Iterate through all peaks
for peak in peaks:
    # Plot all data together
    fig = plt.figure()
    hist_r, bins_r = mg_advanced_plot.energy_plot(energies_r, number_bins, 'Multi-Grid:R',
                                                  start=peak-peak_window/2, stop=peak+peak_window/2,
                                                  color='green', scaling=MONITOR_NORM_R)
    hist_nr, bins_nr = mg_advanced_plot.energy_plot(energies_nr, number_bins, 'Multi-Grid:NR',
                                                    start=peak-peak_window/2, stop=peak+peak_window/2,
                                                    color='blue', scaling=MONITOR_NORM_NR)
    hist_he3, bins_he3 = he3_plot.energy_plot(energies_he3, number_bins, 'Helium-3',
                                              start=peak-peak_window/2, stop=peak+peak_window/2,
                                              color='red', scaling=MONITOR_NORM_HE3)
    plt.legend(title='Detector')
    plt.title('Energy distribution, peak at %.3f meV (%.3f Å)' % (mg_hf.A_to_meV(peak), peak))
    plt.ylabel('Normalized counts')
    #plt.locator_params(axis='y', nbins=5)
    #plt.locator_params(axis='x', nbins=5)
    #plt.yscale('log')
    plt.xscale('linear')
    plt.yscale('linear')
    # Save data
    output_path = '../output/peak_at_%.2f_Å_%.2f_meV.pdf' % (peak, mg_hf.A_to_meV(peak))
    fig.savefig(output_path, bbox_inches='tight')
    plt.close()

##### 3.9 Event distribution

In [ ]:
# Declare parameters
number_bins = 20000
interval = [-0.05, 0.05]

# Plot Helium-3 vs Multi-Grid distribution between events
fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(14)
plt.hist(np.diff(he3.tof.values), bins=number_bins, range=interval,
         zorder=5, label='Helium-3', histtype='step')
plt.hist(np.diff(mg_r.tof.values * 62.5e-9), bins=number_bins, range=interval,
         zorder=5, label='Multi-Grid:R', histtype='step')
plt.grid(True, which='major', linestyle='--', zorder=0)
plt.grid(True, which='minor', linestyle='--', zorder=0)
plt.xlabel('Delta_t (s)')
plt.ylabel('Counts')
plt.yscale('log')
plt.title('Helium-3 vs Multi-Grid:R')
plt.legend()
fig.show()

# Plot zoomed
interval = [0, 0.0002]
number_bins = 1000
fig = plt.figure()
fig.set_figheight(5)
fig.set_figwidth(14)
plt.hist(np.diff(he3.tof.values), bins=number_bins, range=interval,
         zorder=5, label='Helium-3', histtype='step')
plt.hist(np.diff(mg_r.tof.values * 62.5e-9), bins=number_bins, range=interval,
         zorder=5, label='Multi-Grid:R', histtype='step')
plt.grid(True, which='major', linestyle='--', zorder=0)
plt.grid(True, which='minor', linestyle='--', zorder=0)
plt.xlabel('Delta_t (s)')
plt.ylabel('Counts')
plt.yscale('log')
plt.title('Helium-3 vs Multi-Grid:R')
plt.legend()
fig.show()

### 4. Analysis<a class="anchor" id="ANALYSIS"></a>

##### 4.1 Electronic noise<a class="anchor" id="ELECTRONIC_NOISE"></a>

##### 4.2 Efficiency<a class="anchor" id="EFFICIENCY"></a>

###### 4.2.1 Helium-3, calculated and measured efficiency

In [ ]:
%matplotlib notebook

# Declare paths
he3_centered_path = '../tables/he3_efficiency_centered.txt'
he3_5mm_offset_path = '../tables/he3_efficiency_5mm_offset.txt'
he3_incorrect_radius_path = '../tables/He3_efficiency_Full_wrong_radius.txt'
he3_incorrect_radius_path2 = '../tables/He3_efficiency_wrong_radius.txt'
he3_efficiency_vs_radius = '../tables/he3_efficiency_vs_offset.txt'

# Load data
he3_e_center = np.loadtxt(he3_centered_path, delimiter=",", unpack=True)
he3_e_5mm_offset = np.loadtxt(he3_5mm_offset_path, delimiter=",", unpack=True)
he3_e_wrong_full = np.loadtxt(he3_incorrect_radius_path, delimiter=",", unpack=True)
he3_e_wrong = np.loadtxt(he3_incorrect_radius_path2, delimiter=",", unpack=True)
he3_e_vs_radius = np.loadtxt(he3_efficiency_vs_radius, delimiter=",", unpack=True)

# Plot efficiency tube diameter @ 2.5 Å, measured and calculated
positions = np.array([-12, -9, -6, -3, 0, 3, 6, 9, 12])
efficiencies = np.array([49.7, 89.7, 94.5, 96.2, 96.3, 96.2, 95.2, 90.6, 43.7])
uncertainties = np.array([0.2, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.2])
fig = plt.figure()
plt.grid(True, which='major', linestyle='--', zorder=0)
plt.grid(True, which='minor', linestyle='--', zorder=0)
plt.plot(he3_e_vs_radius[0], he3_e_vs_radius[1], label='Calculation', zorder=5)
plt.errorbar(positions/10, efficiencies/100, uncertainties/100, fmt='.', label='Measurement', zorder=5)
plt.legend()
fig.show()

# Plot 
fig = plt.figure()
plt.grid(True, which='major', linestyle='--', zorder=0)
plt.grid(True, which='minor', linestyle='--', zorder=0)
plt.plot(he3_e_center[0], he3_e_center[1], label='Centered', zorder=5)
plt.plot(he3_e_5mm_offset[0], he3_e_5mm_offset[1], label='5 mm offset', zorder=5)
#plt.plot(he3_e_wrong_full[0], he3_e_wrong_full[1], label='Wrong radius', zorder=5)
#plt.plot(he3_e_wrong[0], he3_e_wrong[1], label='Wrong radius', zorder=5)
plt.legend()
fig.show()

###### 4.2.2 Peak areas

In [ ]:
# =============================================================================
#                            GET ALL PEAK AREAS AND FWHM
# =============================================================================

# Calculate energy distribution
energies_nr = mg_energy.get_energies(mg_nr, DIST_OFFSET_NR)
energies_he3 = he3_energy.get_energies(he3, DIST_OFFSET_HE3)

# Define peak locations (here we use the He-3 peak locations)
peaks = np.array([0.886, 1.030, 1.177, 1.321, 1.466, 1.610, 1.755, 1.900, 2.044, 2.188, 
                  2.333, 2.477, 2.622, 2.766, 2.911, 3.056, 3.200, 3.344, 3.489, 3.633,
                  3.778, 3.923, 4.068, 4.211, 4.356, 4.500, 4.645, 4.790, 4.934, 5.079,
                  5.223, 5.368, 5.513, 5.657, 5.801, 5.946, 6.090, 6.235, 6.379, 6.524,
                  6.669, 6.813, 6.958, 7.102, 7.246, 7.391, 7.536, 7.680, 7.825, 7.969])
peak_window = 0.06

# Declare plotting specifications
number_bins = 250
lower_y = 0
upper_y = 40e3

# Declare area integration limits, in units of sigma
peak_lower_limit = -16
peak_upper_limit = 4
background_lower_limit = -30
background_upper_limit = -25

# Prepare data for iteration
energies_vec = [energies_nr, energies_he3]
label_vec = ['Multi-Grid detector:NR', 'Helium-3 tube']
weights_vec = [np.ones(len(energies_nr)), np.ones(len(energies_he3))]
colors = ['blue', 'red']

# Declare data vectors where the peak areas will be saved
peak_areas = [[], []]
peak_widths = [[], []]

# Iterate through all peaks
for peak in peaks[0:50]:
    fig = plt.figure()
    fig.set_figheight(7)
    fig.set_figwidth(14)
    fig.suptitle('Peak @ %.3f meV (%.3f Å)' % (mg_hf.A_to_meV(peak), peak))
    for i, (energies, weights, color, label) in enumerate(zip(energies_vec, weights_vec, colors, label_vec)):
        # Plot data
        plt.subplot(1, 2, i+1)
        hist, bins = mg_advanced_plot.energy_plot(energies, number_bins, label,
                                                  start=peak-peak_window/2, stop=peak+peak_window/2,
                                                  color=color)
        # Get fit parameter guesses
        a_guess, x0_guess, sigma_guess = mg_hf.get_fit_parameters_guesses(hist, bins)
        # Define fit region based on these guesses, ±4 sigma
        left_fit, right_fit = (x0_guess - (4 * sigma_guess)), (x0_guess + (4 * sigma_guess))
        hist_fit, bins_fit = mg_hf.get_hist(energies, number_bins, left_fit, right_fit)
        
        # Fit data and plot fit
        a, x0, sigma, x_fit, y_fit, *_ = mg_hf.fit_data(hist_fit, bins_fit, a_guess, x0_guess, sigma_guess)
        plt.plot(x_fit, y_fit*(max(hist)/max(y_fit)), label='Gaussian fit - %s' % label, color='black')
        # Get peak area
        #bin_width = bins[1] - bins[0]
        #peak_area = mg_hf.get_peak_area(energies, x0, sigma, bin_width, peak_lower_limit, peak_upper_limit,
        #                                background_lower_limit, background_upper_limit, weights=weights)
        
        # Get resolution
        #peak_width = 2 * np.sqrt(2*log(2)) * sigma
        
        # Plot borders
        plt.axvline(x=x0 + peak_lower_limit*sigma, color='orange', linewidth=2,
                    label='Peak border: %d σ' % peak_lower_limit)
        plt.axvline(x=x0 + peak_upper_limit*sigma, color='orange', linewidth=2,
                    label='Peak border: %d σ' % peak_upper_limit)
        
        plt.axvline(x=x0 + background_lower_limit*sigma, color='orange', linewidth=2,
                    label='Background border: %d σ' % background_lower_limit)
        plt.axvline(x=x0 + background_upper_limit*sigma, color='orange', linewidth=2,
                    label='Background border: %d σ' % background_upper_limit)
        
        # Stylize plot
        plt.title(label)
        plt.ylabel('Counts')
        plt.xscale('linear')
        plt.yscale('log')
        plt.legend()
        
        # Store peak area and resolution
        #peak_areas[i].append(peak_area)
        #peak_widths[i].append(peak_width)
        
    
    # Save data
    output_path = '../output/peak_at_%.2f_Å_%.2f_meV.pdf' % (peak, A_to_meV(peak))
    fig.savefig(output_path, bbox_inches='tight')
    plt.close()
    

##### 4.3 Lineshape

##### 4.4 Time- and energy resolution

In [ ]:
# Plot data
fig = mg_advanced_plot.layers_tof(mg_r, 'Radial coating')
fig.show()
fig = mg_advanced_plot.layers_tof(mg_nr, 'No radial coated')
fig.show()

##### 4.5 Parasitic peaks

In [ ]:
# Multi-Grid @ 5.1 Å
start = 35462  # us
end = 36493 # us
peak = 36182 # us
duration_mg = end - start
peak_loc_mg = peak - start
print('--------------------')
print('Multi-Grid')
print('Duration: %d us' % duration_mg)
print('Peak location: %d us' % peak_loc_mg)
print('Relative peak location: %.2f' % (peak_loc_mg/duration_mg))
print('--------------------')

# Helium-3 @ 5.1 Å
start = 35231 # us
end = 36260 # us
peak = 36048 # us
duration_he3 = end - start
peak_loc_he3 = peak - start
print('--------------------')
print('Helium-3')
print('Duration: %d us' % duration_he3)
print('Peak location: %d us' % peak_loc_he3)
print('Relative peak location: %.2f' % (peak_loc_he3/duration_he3))
print('--------------------')



### Acknowledgements

### References